A template of the RL training

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from datetime import datetime

from utils import print_log

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# load the created dataset
dataset_folder_path = Path("dataset", "20250707", "split")

In [4]:
# copied from 03_data_split.ipynb

# Helper functions for the new split folder structure
def load_split_data_from_folder(split_folder, split_type='train'):
    """Load aggregate data from split folder"""
    segments = []
    with open(split_folder / f'{split_type}_segments.txt', 'r') as f:
        for line in f:
            start_str, end_str = line.strip().split(' - ')
            start = datetime.fromisoformat(start_str)
            end = datetime.fromisoformat(end_str)
            segments.append((start, end))
    
    df = pd.read_pickle(split_folder / f'{split_type}_aggregate_df.pkl')
    return segments, df

def load_signatures_from_split_folder(split_folder, split_type, appliance):
    """Load appliance signatures from split folder"""
    sig_path = split_folder / 'load_signature_library' / split_type / appliance / 'load_signatures.pkl'
    ranges_path = split_folder / 'load_signature_library' / split_type / appliance / 'selected_ranges.txt'
    
    if not sig_path.exists():
        return pd.DataFrame(), []
    
    signatures_df = pd.read_pickle(sig_path)
    ranges = []
    if ranges_path.exists():
        with open(ranges_path, 'r') as f:
            for line in f:
                start, end = map(int, line.strip().split(','))
                ranges.append((start, end))
    
    return signatures_df, ranges

In [9]:
# convert datetime objects to timezone-naive datetime objects
def convert_to_naive_datetimes_df(df):
    """Convert datetime objects in DataFrame to timezone-naive datetime objects"""
    df['datetime'] = df['datetime'].apply(lambda x: x.replace(tzinfo=None) if isinstance(x, datetime) else x)

    return df

def convert_to_naive_datetimes(segments):
    """Convert datetime objects in segments to timezone-naive datetime objects"""
    return [(start.replace(tzinfo=None), end.replace(tzinfo=None)) for start, end in segments]

In [8]:
aggregate_load_segments_train, aggregate_load_df_train = load_split_data_from_folder(dataset_folder_path, 'train')
aggregate_load_segments_test, aggregate_load_df_test = load_split_data_from_folder(dataset_folder_path, 'test') 
aggregate_load_segments_validation, aggregate_load_df_validation = load_split_data_from_folder(dataset_folder_path, 'val')

In [10]:
aggregate_load_segments_train, aggregate_load_df_train = convert_to_naive_datetimes(aggregate_load_segments_train), convert_to_naive_datetimes_df(aggregate_load_df_train)
aggregate_load_segments_test, aggregate_load_df_test = convert_to_naive_datetimes(aggregate_load_segments_test), convert_to_naive_datetimes_df(aggregate_load_df_test)
aggregate_load_segments_validation, aggregate_load_df_validation = convert_to_naive_datetimes(aggregate_load_segments_validation), convert_to_naive_datetimes_df(aggregate_load_df_validation)

In [11]:
aggregate_load_df_train

,timestamp,aggregate,datetime,washing_machine,dishwasher,fridge,kettle,microwave,toaster,tv,htpc,gas_oven,kitchen_lights
0,1.357603e+09,234.0,2013-01-08 00:00:05,0.0,1.0,0.0,1.0,1.0,0.0,1.0,69.0,NaN,0.0
1,1.357603e+09,231.0,2013-01-08 00:00:11,0.0,1.0,0.0,1.0,1.0,0.0,1.0,70.0,NaN,0.0
2,1.357603e+09,234.0,2013-01-08 00:00:17,0.0,1.0,0.0,1.0,1.0,0.0,1.0,70.0,NaN,0.0
3,1.357603e+09,232.0,2013-01-08 00:00:23,0.0,1.0,0.0,1.0,1.0,0.0,1.0,68.0,NaN,0.0
4,1.357603e+09,232.0,2013-01-08 00:00:30,0.0,1.0,0.0,1.0,1.0,0.0,1.0,70.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231636,1.388448e+09,178.0,2013-12-30 23:59:35,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0
2231637,1.388448e+09,177.0,2013-12-30 23:59:41,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0
2231638,1.388448e+09,178.0,2013-12-30 23:59:47,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0
2231639,1.388448e+09,178.0,2013-12-30 23:59:53,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,3.0,0.0


In [12]:
aggregate_load_segments_train

[(datetime.datetime(2013, 1, 8, 0, 0),
  datetime.datetime(2013, 1, 10, 23, 59, 59, 999999)),
 (datetime.datetime(2013, 2, 27, 0, 0),
  datetime.datetime(2013, 2, 28, 23, 59, 59, 999999)),
 (datetime.datetime(2013, 3, 8, 0, 0),
  datetime.datetime(2013, 3, 10, 23, 59, 59, 999999)),
 (datetime.datetime(2013, 3, 28, 0, 0),
  datetime.datetime(2013, 3, 31, 23, 59, 59, 999999)),
 (datetime.datetime(2013, 3, 22, 0, 0),
  datetime.datetime(2013, 3, 26, 23, 59, 59, 999999)),
 (datetime.datetime(2013, 4, 8, 0, 0),
  datetime.datetime(2013, 4, 10, 23, 59, 59, 999999)),
 (datetime.datetime(2013, 4, 26, 0, 0),
  datetime.datetime(2013, 4, 30, 23, 59, 59, 999999)),
 (datetime.datetime(2013, 4, 1, 0, 0),
  datetime.datetime(2013, 4, 7, 23, 59, 59, 999999)),
 (datetime.datetime(2013, 5, 15, 0, 0),
  datetime.datetime(2013, 5, 16, 23, 59, 59, 999999)),
 (datetime.datetime(2013, 5, 1, 0, 0),
  datetime.datetime(2013, 5, 7, 23, 59, 59, 999999)),
 (datetime.datetime(2013, 5, 25, 0, 0),
  datetime.dateti

In [ ]:
from rl_env.env_data_loader import SmartMeterDataLoader

sm_dl_train = SmartMeterDataLoader(
    aggregate_load_segments=aggregate_load_segments_train,
    aggregate_load_df=aggregate_load_df_train
)

sm_dl_train.get_divided_segments_length()

162

In [ ]:
sm_dl_train.divided_segments[7]

array([datetime.datetime(2013, 3, 10, 0, 0),
       datetime.datetime(2013, 3, 10, 23, 59, 59, 999999)], dtype=object)

In [ ]:
# sample segment

sm_dl_train.get_aggregate_load_segment(13)

,timestamp,aggregate,datetime
104747,1.363997e+09,335.0,2013-03-23 00:00:05
104748,1.363997e+09,336.0,2013-03-23 00:00:11
104749,1.363997e+09,333.0,2013-03-23 00:00:17
104750,1.363997e+09,334.0,2013-03-23 00:00:24
104751,1.363997e+09,331.0,2013-03-23 00:00:30
...,...,...,...
118501,1.364083e+09,179.0,2013-03-23 23:59:30
118502,1.364083e+09,171.0,2013-03-23 23:59:37
118503,1.364083e+09,171.0,2013-03-23 23:59:43
118504,1.364083e+09,171.0,2013-03-23 23:59:49


In [33]:
# create dataloader for validation and test sets
sm_dl_validation = SmartMeterDataLoader(
    aggregate_load_segments=aggregate_load_segments_validation,
    aggregate_load_df=aggregate_load_df_validation
)

sm_dl_test = SmartMeterDataLoader(
    aggregate_load_segments=aggregate_load_segments_test,
    aggregate_load_df=aggregate_load_df_test
)

(Optional) Load the pre-trained H-network and related components

In final product, the H-network should be trained along with the DDQL/PPO agent

In [16]:
import torch
from model.H_network.h_network import HNetwork

h_network_datetime = datetime(2025, 7, 12)

h_network_path = Path("model_trained", f"h_network_{h_network_datetime.strftime('%Y%m%d')}.pth")

h_network = HNetwork(2, 44, 1)
h_network.load_state_dict(torch.load(h_network_path))
h_network.eval()

HNetwork(
  (LSTM_1): LSTM(2, 44, batch_first=True, bidirectional=True)
  (ac1): Tanh()
  (LSTM_2): LSTM(88, 1, batch_first=True, bidirectional=True)
  (ac2): Tanh()
  (fc): Linear(in_features=2, out_features=1, bias=True)
)

In [17]:
h_network_stdscaler_path = Path("model_trained", f"h_network_standardscaler_{h_network_datetime.strftime('%Y%m%d')}.pkl")
import joblib
h_network_stdscaler = joblib.load(h_network_stdscaler_path)

Create the environment

In [ ]:
import sys
sys.path.append(str(Path('rl_env')))

from rl_env.hrl_env import SmartMeterWorld

env_train = SmartMeterWorld(
    smart_meter_data_loader=sm_dl_train,
    render_mode="human",
)

env_train.set_h_network(h_network)
env_train.set_h_network_stdscaler(h_network_stdscaler)

[2025-07-12 16:14:44:217] [SmartMeterWorld] Render mode set to 'human'. Render server at 127.0.0.1:50007. render_connected: True. render_client_socket: <socket.socket fd=92, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 41422), raddr=('127.0.0.1', 50007)>


In [ ]:
from gymnasium.utils.env_checker import check_env

# This will catch many common issues
try:
    check_env(env_train)
    print("Environment passes all checks!")
except Exception as e:
    print(f"Environment has issues: {e}")

[2025-07-12 14:57:51:161] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 13864, 'datetime_range': (Timestamp('2013-07-01 00:00:03'), Timestamp('2013-07-01 23:59:54'))}
[2025-07-12 14:57:51:172] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 13880, 'datetime_range': (Timestamp('2013-08-28 00:00:02'), Timestamp('2013-08-28 23:59:59'))}
[2025-07-12 14:57:51:180] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 13745, 'datetime_range': (Timestamp('2013-03-28 00:00:05'), Timestamp('2013-03-28 23:59:57'))}
[2025-07-12 14:57:51:190] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 13880, 'datetime_range': (Timestamp('2013-08-28 00:00:02'), Timestamp('2013-08-28 23:59:59'))}
[2025-07-12 14:57:51:200] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 13745, 'datetime_range': (Timestamp('2013-03-28 00:00:05'), Times

/root/miniforge3/envs/smartmeter/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:334: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
/root/miniforge3/envs/smartmeter/lib/python3.9/site-packages/gymnasium/utils/env_checker.py:434: UserWarning: WARN: Not able to test alternative render modes due to the environment not having a spec. Try instantiating the environment through `gymnasium.make`
  logger.warn(


In [ ]:
obs = env_train.reset()
obs

[2025-07-12 14:57:57:845] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 13745, 'datetime_range': (Timestamp('2013-03-28 00:00:05'), Timestamp('2013-03-28 23:59:57'))}


({'aggregate_load': array([176.], dtype=float32),
  'battery_soc': array([0.22035988], dtype=float32),
  'timestamp_features': array([-0.5 ,  0.  , -0.25], dtype=float32)},
 {})

In [ ]:
env_train.reset_render_window()

In [ ]:
# initialize a PPO agent
from stable_baselines3 import PPO

rl_datetime = datetime.now()
tensorboard_log_path = Path("rl_model", "PPO", f"{rl_datetime.strftime('%Y%m%d_%H%M%S')}")

rl_model = PPO(
    "MultiInputPolicy", 
    env_train, 
    verbose=2,
    tensorboard_log=tensorboard_log_path
)

rl_model.learn(
    total_timesteps=100000,
    progress_bar=True,
    tb_log_name="PPO_SmartMeterWorld"
)

2025-07-12 14:58:07.867761: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-12 14:58:07.999517: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752328688.046938  448068 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752328688.062176  448068 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752328688.170229  448068 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
[2025-07-12 14:58:10:488] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 13955, 'datetime_range': (Timestamp('2013-06-29 00:00:03'), Timestamp('2013-06-29 23:59:55'))}
Logging to rl_model/PPO/20250712_145809/PPO_SmartMeterWorld_1


Output()

-----------------------------
| time/              |      |
|    fps             | 86   |
|    iterations      | 1    |
|    time_elapsed    | 23   |
|    total_timesteps | 2048 |
-----------------------------


------------------------------------------
| time/                   |              |
|    fps                  | 77           |
|    iterations           | 2            |
|    time_elapsed         | 52           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0024565784 |
|    clip_fraction        | 0.00801      |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.42        |
|    explained_variance   | 1.55e-06     |
|    learning_rate        | 0.0003       |
|    loss                 | 6.36e+05     |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.000994    |
|    std                  | 0.995        |
|    value_loss           | 1.45e+06     |
------------------------------------------


...

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1.38e+04      |
|    ep_rew_mean          | 7.57e+06      |
| time/                   |               |
|    fps                  | 56            |
|    iterations           | 46            |
|    time_elapsed         | 1680          |
|    total_timesteps      | 94208         |
| train/                  |               |
|    approx_kl            | 0.00062993076 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.41         |
|    explained_variance   | 5.96e-08      |
|    learning_rate        | 0.0003        |
|    loss                 | 5.55e+07      |
|    n_updates            | 450           |
|    policy_gradient_loss | -0.000352     |
|    std                  | 0.986         |
|    value_loss           | 1.1e+08       |
-------------------------------------------


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1.38e+04      |
|    ep_rew_mean          | 7.57e+06      |
| time/                   |               |
|    fps                  | 55            |
|    iterations           | 47            |
|    time_elapsed         | 1728          |
|    total_timesteps      | 96256         |
| train/                  |               |
|    approx_kl            | 0.00018839748 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.4          |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | 8.83e+07      |
|    n_updates            | 460           |
|    policy_gradient_loss | -0.000143     |
|    std                  | 0.985         |
|    value_loss           | 1.76e+08      |
-------------------------------------------


[2025-07-12 15:27:16:130] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 
13711, 'datetime_range': (Timestamp('2013-05-01 00:00:01'), Timestamp('2013-05-01 23:59:59'))}

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.38e+04     |
|    ep_rew_mean          | 7.55e+06     |
| time/                   |              |
|    fps                  | 55           |
|    iterations           | 48           |
|    time_elapsed         | 1758         |
|    total_timesteps      | 98304        |
| train/                  |              |
|    approx_kl            | 0.0011467711 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.4         |
|    explained_variance   | -1.19e-07    |
|    learning_rate        | 0.0003       |
|    loss                 | 1.36e+08     |
|    n_updates            | 470          |
|    policy_gradient_loss | -0.00105     |
|    std                  | 0.984        |
|    value_loss           | 2.72e+08     |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.38e+04     |
|    ep_rew_mean          | 7.55e+06     |
| time/                   |              |
|    fps                  | 56           |
|    iterations           | 49           |
|    time_elapsed         | 1783         |
|    total_timesteps      | 100352       |
| train/                  |              |
|    approx_kl            | 0.0011077211 |
|    clip_fraction        | 9.77e-05     |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.4         |
|    explained_variance   | 0            |
|    learning_rate        | 0.0003       |
|    loss                 | 4.94e+07     |
|    n_updates            | 480          |
|    policy_gradient_loss | -0.000471    |
|    std                  | 0.981        |
|    value_loss           | 9.96e+07     |
------------------------------------------


Create a testing environment

In [ ]:
obs, info = env_train.reset()
for i in range(1):
    done = False
    while not done:
        action, _states = rl_model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env_train.step(action)
        print_log(f"Step: {env_train.episode.get_current_step()}, Action: {action}, Reward: {reward}")
        env_train.render()

[2025-07-12 16:08:33:089] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 13636, 'datetime_range': (Timestamp('2013-12-03 00:00:02'), Timestamp('2013-12-03 23:59:58'))}
[2025-07-12 16:08:33:095] Step: 1, Action: [-0.11176914], Reward: 0.3195078111757835
[2025-07-12 16:08:33:107] Step: 2, Action: [-0.11176719], Reward: 0.3444152867542605
[2025-07-12 16:08:33:117] Step: 3, Action: [-0.11176597], Reward: 0.20051824648367367
[2025-07-12 16:08:33:126] Step: 4, Action: [-0.11176914], Reward: 0.1035867738970452
[2025-07-12 16:08:33:133] Step: 5, Action: [-0.11177026], Reward: 0.05550562221085032
[2025-07-12 16:08:33:139] Step: 6, Action: [-0.11176758], Reward: 0.04985590043287724
[2025-07-12 16:08:33:145] Step: 7, Action: [-0.11177026], Reward: 0.05582076873700652
[2025-07-12 16:08:33:154] Step: 8, Action: [-0.11176875], Reward: 0.07269255363021294
[2025-07-12 16:08:33:160] Step: 9, Action: [-0.11176836], Reward: 0.0971282958035469
[2025-07-12 16:08:33:166

In [ ]:
env_train.episode.df

,timestamp,aggregate,datetime,grid_load,battery_soc
0,1.357603e+09,234.0,2013-01-08 00:00:05,None,None
1,1.357603e+09,231.0,2013-01-08 00:00:11,None,None
2,1.357603e+09,234.0,2013-01-08 00:00:17,None,None
3,1.357603e+09,232.0,2013-01-08 00:00:23,None,None
4,1.357603e+09,232.0,2013-01-08 00:00:30,None,None
...,...,...,...,...,...
13375,1.357690e+09,178.0,2013-01-08 23:59:33,None,None
13376,1.357690e+09,177.0,2013-01-08 23:59:39,None,None
13377,1.357690e+09,178.0,2013-01-08 23:59:46,None,None
13378,1.357690e+09,177.0,2013-01-08 23:59:52,None,None


In [ ]:
# save the graph

env_train.save_graph(
    str(Path("rl_model", "PPO", f"{rl_datetime.strftime('%Y%m%d_%H%M%S')}", "graph_train.png"))
)

In [ ]:
env_train.close()

[2025-07-12 04:44:05:356] [SmartMeterWorld] Environment closed.


In [30]:
# save the model
rl_model_path = Path("rl_model", "PPO", f"{rl_datetime.strftime('%Y%m%d_%H%M%S')}", "rl_model.zip")
rl_model.save(rl_model_path)

---

In [34]:
# load the model & environment
import sys
sys.path.append(str(Path('rl_env')))

from rl_env.hrl_env import SmartMeterWorld
from stable_baselines3 import PPO

env_test = SmartMeterWorld(
    sm_dl_test,
    render_mode="human",
)

env_test.set_h_network(h_network)
env_test.set_h_network_stdscaler(h_network_stdscaler)

rl_model_path = Path("rl_model", "PPO", f"{datetime(2025,7,12,14,58,9).strftime('%Y%m%d_%H%M%S')}", "rl_model.zip")
rl_model_loaded = PPO.load(rl_model_path, env=env_test)

[2025-07-12 17:01:03:406] [SmartMeterWorld] Render mode set to 'human'. Render server at 127.0.0.1:50007. render_connected: True. render_client_socket: <socket.socket fd=94, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 55314), raddr=('127.0.0.1', 50007)>
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [35]:
obs, info = env_test.reset()
for i in range(1):
    done = False
    while not done:
        action, _states = rl_model_loaded.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env_test.step(action)
        print_log(f"Step: {env_test.episode.get_current_step()}, Action: {action}, Reward: {reward}")
        env_test.render()

[2025-07-12 17:01:09:418] [SmartMeterWorld] Resetting environment with a new episode. Episode info: {'length': 13838, 'datetime_range': (Timestamp('2013-08-15 00:00:01'), Timestamp('2013-08-15 23:59:59'))}
[2025-07-12 17:01:09:427] Step: 1, Action: [-0.11162357], Reward: 0.3197169332109988
[2025-07-12 17:01:09:436] Step: 2, Action: [-0.11163278], Reward: 0.5596025769556463
[2025-07-12 17:01:09:456] Step: 3, Action: [-0.11163721], Reward: 0.48428668455776075
[2025-07-12 17:01:09:464] Step: 4, Action: [-0.11163279], Reward: 0.34103424595346055
[2025-07-12 17:01:09:470] Step: 5, Action: [-0.11162825], Reward: 0.21234606818104784
[2025-07-12 17:01:09:476] Step: 6, Action: [-0.11163279], Reward: 0.12094011353005966
[2025-07-12 17:01:09:487] Step: 7, Action: [-0.11163279], Reward: 0.06497681485166153
[2025-07-12 17:01:09:495] Step: 8, Action: [-0.1116328], Reward: 0.03361730095612258
[2025-07-12 17:01:09:501] Step: 9, Action: [-0.1116328], Reward: 0.017724320196613668
[2025-07-12 17:01:09:50

In [40]:
env_test.save_graph(
    str(Path("rl_model", "PPO", f"{rl_datetime.strftime('%Y%m%d_%H%M%S')}", "graph_test.png"))
)

In [41]:
env_test.close()

[2025-07-12 17:07:43:300] [SmartMeterWorld] Environment closed.
